# GMAG vs Astroquery - Image Download Time Comparison

This notebook compares the galaxy image search and download time between the GMAG and Astroquery packages. 

To run this notebook yourself, download this notebook and the `example_data` folder, and install `gmag` and `astroquery` packages. 

## Setup

- __Catalog__: Galaxy Zoo merging galaxies ([link here](https://data.galaxyzoo.org/data/mergers/darg_mergers.fits)), uses the __first ten rows__ of data (fits file at `example_data/darg_mergers_10.fits`). 
    
- __Number of Images__: 30 (Download SDSS g,r,i band, total of 10x3=30 images).

## Astroquery

In [1]:
# Optional: Clear astroquery cache
import shutil
import pathlib
cache_dir = pathlib.Path.home() / '.astropy' / 'cache' 
if cache_dir.exists():
    shutil.rmtree(cache_dir)
    print('Cleared astroquery cache')

Cleared astroquery cache


In [2]:
import astroquery
from astroquery.sdss import SDSS
from astropy.table import Table, vstack
from astropy import coordinates

print(f"astroquery version: {astroquery.__version__}")

astroquery version: 0.4.6


In [3]:
%%time
# Read table and coordinates
table = Table.read("example_data/darg_mergers_10.fits")
coords = coordinates.SkyCoord(table['ra1'], table['dec1'], unit='deg')

# Search and get images
xid = vstack([SDSS.query_region(c, radius='1m')[0] for c in coords])
imgs = SDSS.get_images(matches=xid, band='gri')

# Store images
pathlib.Path('data').mkdir(parents=True, exist_ok=True)
for i, img in enumerate(imgs):
    rowid = i // 3
    file_name = f"{xid['objid'][rowid]}-{img[0].header['FILTER']}.fits"
    img.writeto(pathlib.Path.cwd() / 'data' / file_name)

CPU times: user 9.06 s, sys: 2.01 s, total: 11.1 s
Wall time: 1min 57s


## GMAG

In [4]:
import gmag
from gmag import sdss

print(f"gmag version: {gmag.__version__}")

gmag version: 2.0.1


In [7]:
%%time
sdss.download_images(
    "example_data/darg_mergers_10.fits", 
    bands='gri', ra_col="ra1", dec_col="dec1", cutout=False
)

Searching galaxies: 100%|██████████████████████| 10/10 [00:01<00:00,  7.28obj/s]

...Found 10 out of 10 galaxies
...Created directories for images at /Users/junyu/Documents/GitHub/GMAG/notebooks/images_2022-11-27_12-09-46


...Saving info file at /Users/junyu/Documents/GitHub/GMAG/notebooks/images_2022-11-27_12-09-46/info.csv
ALL DONE!
CPU times: user 76.9 ms, sys: 139 ms, total: 216 ms
Wall time: 18.9 s


##  6 TIMES FASTER!